In [5]:
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import glob
import os
import time

In [6]:
#base = "/Users/ivankoh/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/1D/NUS Y3S1/BT4012/Data/"
#train_path = base + "twitter_data_train_multiclass.csv" 
train_path = "/Users/ivankoh/Documents/GitHub/Fake-Twitter-Account-Detection/data/twitter_data_train_multiclass.csv"
df = pd.read_csv(train_path)

In [12]:
df[df.duplicated('screen_name')]
test[test.duplicated('screen_name')]

,Unnamed: 0,id,name,screen_name,statuses_count,followers_count,friends_count,favourites_count,listed_count,url,...,description_92,description_93,description_94,description_95,description_96,description_97,description_98,description_99,has_face,account_type_multi


## Get to the correct data format

In [18]:
to_drop = [
    'profile_image_url',
    'profile_use_background_image',
    'profile_background_tile',
    'account_type', 
    'account_type_multi',
    'id', 
    'name', 
    'screen_name',
    'profile_image_url',
    'protected',
    'verified',
    'description',
    'description_processed',
    'tweets_list',
    'tweets_list_processed',
    'url'
    ]

X_train, y_train = df.drop(to_drop, axis=1), df['account_type_multi']

In [19]:
df.columns[df.isna().any()]
a = df.drop(to_drop, axis=1)
a.columns.to_series()[np.isinf(a).any()]


Series([], dtype: object)

## Grid Search with K-fold CV applied to SVC

5 folds 
- approx 311 per fold 
- 62 validation data 
- 249 train data

### 1. Training

In [20]:
# Hyper-parameters
gamma = [1e-7,1e-4,1e-3,1e-1,1e0,1e1]
C = [1e-3,1e-2,1e0,1e1,1e3,1e5]
kernels = ['rbf', 'linear'] #poly takes forever to train 'sigmoid'

params = {'C': C, 
          'gamma': gamma, 
          'kernel': kernels,
          'decision_function_shape':['ovr']
         }

cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=2022)

svc_randomcv = RandomizedSearchCV(
    estimator=SVC(max_iter=100),
    param_distributions=params,
    n_iter=10, 
    cv=cv_method, # k-fold cv
    verbose=2,
    random_state=2022,
    #n_jobs=-1, # use all processors
    scoring='f1_weighted'
)
svc_randomcv.fit(X_train, y_train)

Fitting 15 folds for each of 10 candidates, totalling 150 fits


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END C=100000.0, decision_function_shape=ovr, gamma=1.0, kernel=rbf; total time=   1.0s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END C=100000.0, decision_function_shape=ovr, gamma=1.0, kernel=rbf; total time=   0.9s


...

[CV] END C=0.001, decision_function_shape=ovr, gamma=0.0001, kernel=linear; total time=   0.2s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END C=0.001, decision_function_shape=ovr, gamma=0.0001, kernel=linear; total time=   0.2s


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/sklearn/svm/_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


RandomizedSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=2022),
                   estimator=SVC(max_iter=100),
                   param_distributions={'C': [0.001, 0.01, 1.0, 10.0, 1000.0,
                                              100000.0],
                                        'decision_function_shape': ['ovr'],
                                        'gamma': [1e-07, 0.0001, 0.001, 0.1,
                                                  1.0, 10.0],
                                        'kernel': ['rbf', 'linear']},
                   random_state=2022, scoring='f1_weighted', verbose=2)

### 2. Best Hyper-parameters

In [21]:
print(svc_randomcv.best_params_)
print(svc_randomcv.best_score_)

{'kernel': 'rbf', 'gamma': 0.0001, 'decision_function_shape': 'ovr', 'C': 100000.0}
0.44230226806624134


### 3. Testing

In [11]:
#test_path = base + "twitter_data_test_multiclass.csv"
test_path = "/Users/ivankoh/Documents/GitHub/Fake-Twitter-Account-Detection/data/twitter_data_test_multiclass.csv"
test = pd.read_csv(test_path)

X_test, y_test = test.drop(to_drop, axis=1), test['account_type_multi']

NameError: name 'to_drop' is not defined

In [8]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(9446, 223)
(9446,)
(1667, 223)
(1667,)


In [9]:
best_clf = SVC(**svc_randomcv.best_params_)
target_names = y_train

start_time = time.time()
best_clf.fit(X_train, y_train)
y_pred_test = best_clf.predict(X_test)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds

print(classification_report(y_test, y_pred_test, digits=5))
#roc_auc_score(y_test, y_pred_test)

Total time taken for the program execution 19.212857007980347
              precision    recall  f1-score   support

           0    0.60187   0.93705   0.73295       413
           1    0.95413   0.90435   0.92857       460
           2    1.00000   0.18966   0.31884        58
           3    0.96029   0.79880   0.87213       666
           4    0.55556   0.18868   0.28169        53
           5    1.00000   0.29412   0.45455        17

    accuracy                        0.81644      1667
   macro avg    0.84531   0.55211   0.59812      1667
weighted avg    0.85871   0.81644   0.81094      1667



In [10]:
set(df['account_type'])

{0, 1}